In [1]:
import environs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common import exceptions
from selenium.webdriver.common.action_chains import ActionChains
from datetime import date
import time
from pprint import pprint
import pandas as pd

In [2]:
# goods_category = input('Укажитенаименование товара:\n')
goods_category = 'Фитнес браслет'
driver = webdriver.Chrome(executable_path='./chromedriver')
driver.implicitly_wait(10)
driver.fullscreen_window()
driver.get('https://www.wildberries.ru/')

In [3]:
elem = driver.find_element(By.CLASS_NAME, 'nav-element__search')
elem.click()

In [4]:
elem = driver.find_element(By.ID, 'mobileSearchInput')
elem.send_keys(goods_category)

In [5]:
elem.send_keys(Keys.ENTER)

In [6]:
# goods = driver.find_elements(By.CLASS_NAME, 'product-card')

In [7]:
goods_list = []
# while True:
# Это если вдруг нужны все товары со всех страниц
for j in range(5):
    
    goods = driver.find_elements(By.CLASS_NAME, 'product-card')
    
    for i in range(len(goods)):
        if i > 0:
            goods = driver.find_elements(By.CLASS_NAME, 'product-card')
        try:
            goods[i].click()
        except exceptions.ElementNotInteractableException:
            # На последнем товаре на странице либо это исключение
            break
        time.sleep(3)
        goods_data = {}
        try:
            brand = driver.find_elements(By.XPATH, '//span[@data-link="text{:product^brandName}"]')[1].text
        except IndexError:
            # Либо IndexError
            break
        name = driver.find_elements(By.XPATH, '//span[@data-link="text{:product^goodsName}"]')[1].text
        price = driver.find_element(By.CLASS_NAME, 'price-block__price-wrap').text
        price = price.split('\n')
        seller = driver.find_element(By.CLASS_NAME, 'seller__name').text
        number_sales = driver.find_elements(By.XPATH, '//span[contains(@data-link, "productCardOrderCount")]')[1].text
        link = driver.current_url
        raiting = driver.find_elements(By.XPATH, '//span[@data-link="text{: product^star}"]')[1].text
        reviews_number = driver.find_elements(By.CLASS_NAME, 'same-part-kt__count-review')[1].text
        goods_data['Наименование товара'] = name
        goods_data['Брэнд'] = brand
        try:
            goods_data['Цена со скидкой'] = price[0]
        except IndexError:
            goods_data['Цена со скидкой'] = None
        try:
            goods_data['Цена без скидки'] = price[1]
        except IndexError:
            goods_data['Цена без скидки'] = price[0]
        goods_data['Продавец'] = seller
        goods_data['Количество продаж'] = number_sales
        goods_data['Ссылка на товар'] = link
        goods_data['Рейтинг товара'] = raiting
        goods_data['Количество отзывов'] = reviews_number
        goods_list.append(goods_data)
        next_good = driver.find_element(By.CLASS_NAME, 'sw-slider-kt-mix__btn')
        next_good.click()
        time.sleep(3)
        
    try:
        next_page = driver.find_elements(By.CLASS_NAME, 'pagination__next')[0].get_attribute('href')
        driver.get(next_page)
        time.sleep(3)
    except IndexError:
        # Если while True на последней странице срабатывает данное исключение
        break

In [12]:
n = 0
while True:
    try:
        next_page = driver.find_elements(By.CLASS_NAME, 'pagination__next')[0].get_attribute('href')
        driver.get(next_page)
        time.sleep(3)
    except IndexError:
        break
    n += 1

print(n)

60


In [8]:
len(goods_list)    

504

In [9]:
df = pd.DataFrame(goods_list) 
df.tail()

,Наименование товара,Брэнд,Цена со скидкой,Цена без скидки,Продавец,Количество продаж,Ссылка на товар,Рейтинг товара,Количество отзывов
499,Фитнес-браслет / смарт часы / умные часы / фит...,BandRate Smart,5 805 ₽,13 500 ₽,ИП Солнцев Сергей Васильевич,,https://www.wildberries.ru/catalog/27611180/de...,0,0 отзывов
500,Фитнес-браслет / смарт часы для женщин / умные...,BandRate Smart,4 613 ₽,10 485 ₽,ИП Солнцев Сергей Васильевич,,https://www.wildberries.ru/catalog/27650648/de...,0,0 отзывов
501,Фитнес-браслет,EPSAR,574 ₽,820 ₽,ИП Муцольгов Микаил Тарханович,более 10 раз,https://www.wildberries.ru/catalog/33883701/de...,2,7 отзывов
502,"Умные часы, профессиональная серия. ЭКГ, давле...",37degree,4 454 ₽,16 000 ₽,ИП Смирнов Олег Сергеевич,более 200 раз,https://www.wildberries.ru/catalog/15960301/de...,4,41 отзыв
503,Фитнес-браслет M6 русская прошивка с магнитной...,Фитнес браслет,525 ₽,1 788 ₽,ИП Ацапин Денис Алексеевич,более 40 раз,https://www.wildberries.ru/catalog/41933995/de...,4,7 отзывов


In [10]:
df.to_excel('men_running_shoes.xlsx')